In [1]:
# https://www.properati.com.ar/data
#! wget -O ar_properties.csv.gz -q https://storage.googleapis.com/properati-data-public/ar_properties.csv.gz
#! gzip -d -f ./ar_properties.csv.gz

In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

import sklearn as sk
from sklearn import model_selection
from sklearn import ensemble
from sklearn import metrics

import datetime

# Lectura de datos

In [3]:
df_properties = pd.read_csv('../ar_properties.csv', index_col="id")
df_apred = pd.read_csv('../a_predecir.csv', index_col="id")

In [4]:
df = df_properties.copy()
df_pred = df_apred.copy()

In [5]:
df.head()

,ad_type,start_date,end_date,created_on,lat,lon,l1,l2,l3,l4,...,bathrooms,surface_total,surface_covered,price,currency,price_period,title,description,property_type,operation_type
id,,,,,,,,,,,,,,,,,,,,,
DyVXfkpKygVBKuUk5olH+A==,Propiedad,2020-08-22,2020-09-03,2020-08-22,-34.407468,-58.957367,Argentina,Bs.As. G.B.A. Zona Norte,Pilar,NaN,...,NaN,133139.0,NaN,NaN,NaN,Mensual,VENTA - Lote Industrial 130.000m2 - Parque Ind...,Lote (Nro.50) mide 133.139 m2\n<br>Valor: U$S ...,Lote,Venta
9naojilaMecJN4jlQiTkGg==,Propiedad,2020-08-22,2020-09-04,2020-08-22,-37.996039,-57.542509,Argentina,Buenos Aires Costa Atlántica,Mar del Plata,NaN,...,NaN,687.0,687.0,NaN,NaN,Mensual,Casa - Santa Cecilia,Conjunto edilicio de casa y departamentos en b...,Otro,Venta
tlCPRJPjoDEUzuuCelemAQ==,Propiedad,2020-08-22,2020-08-31,2020-08-22,-31.380187,-58.009182,Argentina,Entre Ríos,Concordia,NaN,...,1.0,80.0,80.0,NaN,NaN,Mensual,Casa - Concordia,"Casa de un dormitorio, cocina, baño, living co...",Casa,Alquiler
Zw3b91glQUO3HNrM5fPYlQ==,Propiedad,2020-08-22,2020-09-04,2020-08-22,-27.494106,-55.123455,Argentina,Misiones,Oberá,NaN,...,1.0,NaN,NaN,NaN,NaN,Mensual,Terreno - Obera,IMPORTANTE PROPIEDAD EN VENTA EN OBERA MISIONE...,Lote,Venta
bsU81gm9JEgtZCbTYgvykg==,Propiedad,2020-08-22,2020-09-04,2020-08-22,-32.948856,-60.630464,Argentina,Santa Fe,Rosario,NaN,...,1.0,76.0,66.0,NaN,NaN,Mensual,Departamento - Rosario,PORTAL DE LOS MARINOS<br>Es un complejo de viv...,Departamento,Venta


In [6]:
df_pred.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1087 entries, 1 to 1087
Data columns (total 27 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ad_type          1087 non-null   object 
 1   start_date       1087 non-null   object 
 2   end_date         1087 non-null   object 
 3   created_on       1087 non-null   object 
 4   lat              1087 non-null   float64
 5   lon              1087 non-null   float64
 6   l1               1087 non-null   object 
 7   l2               1087 non-null   object 
 8   l3               1087 non-null   object 
 9   rooms            1083 non-null   float64
 10  bedrooms         958 non-null    float64
 11  bathrooms        1083 non-null   float64
 12  surface_total    1087 non-null   float64
 13  surface_covered  1087 non-null   float64
 14  price            0 non-null      float64
 15  currency         1087 non-null   object 
 16  price_period     0 non-null      float64
 17  title         

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1000000 entries, DyVXfkpKygVBKuUk5olH+A== to rEynGyGrrsxNeG0ib5vUBA==
Data columns (total 24 columns):
 #   Column           Non-Null Count    Dtype  
---  ------           --------------    -----  
 0   ad_type          1000000 non-null  object 
 1   start_date       1000000 non-null  object 
 2   end_date         1000000 non-null  object 
 3   created_on       1000000 non-null  object 
 4   lat              894233 non-null   float64
 5   lon              894191 non-null   float64
 6   l1               1000000 non-null  object 
 7   l2               1000000 non-null  object 
 8   l3               965273 non-null   object 
 9   l4               306162 non-null   object 
 10  l5               5530 non-null     object 
 11  l6               0 non-null        float64
 12  rooms            714179 non-null   float64
 13  bedrooms         649933 non-null   float64
 14  bathrooms        765122 non-null   float64
 15  surface_total    477831 non-nul

# Tratamiento de nulls y outliers

## Longitud y Latitud

In [8]:
df = df.loc[df["l1"] == "Argentina"]

In [9]:
df = df.loc[df["l2"] == "Capital Federal"]

Coordenadas maximas CABA: \
Norte - (-34.531639, -58.464800)\
Este - (-34.62073349940953, -58.338011418419356)\
Oeste - (-34.61568853226109, -58.532674897063565)\
Sur - (-34.7058110012102, -58.4602477739537)\
\
Limites:\
Longitud - (-58.532674897063565, -58.338011418419356 )\
Latitud - (-34.7058110012102, -34.531639)

In [10]:
lon_min_max = (-58.532674897063565, -58.338011418419356 )
lat_min_max = (-34.7058110012102, -34.531639)

In [11]:
df_lon_lat = df.loc[df["l2"] == "Capital Federal"]

In [12]:
df_lon_lat.drop(df_lon_lat.loc[df_lon_lat["lat"].isnull() & df_lon_lat["lon"].isnull() & df_lon_lat["l3"].isnull()].index, inplace = True)
df_lon_lat = df_lon_lat[["lat", "lon", "l3"]]
df_lon_lat["lat_outlier"] = df_lon_lat.lat.apply(lambda x: 0 if (x > lat_min_max[0]) & (x < lat_min_max[1]) else 1  )
df_lon_lat["lon_outlier"] = df_lon_lat.lon.apply(lambda x: 0 if (x > lon_min_max[0]) & (x < lon_min_max[1]) else 1 )

In [13]:
df_lon_lat.drop(df_lon_lat.loc[(df_lon_lat.lat_outlier == 1) & (df_lon_lat.l3.isnull())].index, inplace = True)
out_lon_lat_ind = df_lon_lat.loc[(df_lon_lat.lat_outlier == 1) | (df_lon_lat.lon_outlier == 1)].index
df_lon_lat_out = df_lon_lat.loc[(df_lon_lat.lat_outlier == 1) | (df_lon_lat.lon_outlier == 1)]
df_lon_lat = df_lon_lat.loc[(df_lon_lat.lat_outlier == 0) & (df_lon_lat.lon_outlier == 0)]

In [14]:
l3_list = list(df_lon_lat.l3.unique())
l3_dict_lat = {}
l3_dict_lon = {}
for i in l3_list:
    l3_dict_lon[i] = df_lon_lat.loc[df_lon_lat.l3 == i, "lon"].mean()
    l3_dict_lat[i] = df_lon_lat.loc[df_lon_lat.l3 == i, "lat"].mean()

df_lon_lat_out["lon"] = df_lon_lat_out.l3.apply(lambda x:l3_dict_lon[x])
df_lon_lat_out["lat"] = df_lon_lat_out.l3.apply(lambda x:l3_dict_lat[x])

In [15]:
for ind in out_lon_lat_ind:
    df["lon"].loc[ind] = df_lon_lat_out["lon"].loc[ind]
    df["lat"].loc[ind] = df_lon_lat_out["lat"].loc[ind]   

/tmp/ipykernel_84058/3971525020.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["lon"].loc[ind] = df_lon_lat_out["lon"].loc[ind]
/tmp/ipykernel_84058/3971525020.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["lat"].loc[ind] = df_lon_lat_out["lat"].loc[ind]


## l3 - Barrio

Corregidas las longitudes y latitudes generamos un knn para predecir los barrios y completar l3

In [16]:
df_l3 = df[["lon","lat","l3"]]

df_l3.drop(df_l3.loc[df_l3["lon"].isnull()].index,inplace = True)
df_l3.drop(df_l3.loc[df_l3["lat"].isnull()].index,inplace = True)

df_test_l3 = df_l3.loc[df_l3["l3"].isnull()]

df_train_l3 = df_l3.loc[df_l3["l3"].notnull()]

/tmp/ipykernel_84058/2478595951.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_l3.drop(df_l3.loc[df_l3["lon"].isnull()].index,inplace = True)
/tmp/ipykernel_84058/2478595951.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_l3.drop(df_l3.loc[df_l3["lat"].isnull()].index,inplace = True)


In [17]:
X_l3 = df_train_l3[df_train_l3.columns.drop('l3')]
y_l3 = df_train_l3['l3']

# Partimos en entrenamiento y prueba 

for i in [1,3,5,7,9,11]:
    # Creamos el objeto del modelo
    neigh = sk.neighbors.KNeighborsClassifier(n_neighbors=i, n_jobs = -1)
    
    kf = sk.model_selection.KFold(n_splits=10, shuffle=True, random_state=42)
    
    scores = []
    
    for train, test in kf.split(X_l3):
        
        X_train, X_test, y_train, y_test = X_l3.iloc[train], X_l3.iloc[test], y_l3[train], y_l3[test]
    
        # Entrenamos el modelo
        neigh.fit(X_train, y_train)
    
        # Predecimos
        y_pred_l3 = neigh.predict(X_test)
    
        # Medimos la performance de la predicción
        score = sk.metrics.accuracy_score(y_test, y_pred_l3)
        scores.append(score)
        #print(score)
    print(f"n_{i} {np.mean(scores)}")


n_1 0.8380804193175351
n_3 0.8383278341978697
n_5 0.8398275092636194
n_7 0.8394354615245071
n_9 0.8387389094042674
n_11 0.8381451235140739


In [18]:
X_pred_l3 = df_test_l3[df_test_l3.columns.drop('l3')]

# Entrenamiento
neigh = sk.neighbors.KNeighborsClassifier(n_neighbors = 5, n_jobs = -1)

# Entrenamos el modelo
neigh.fit(X_train, y_train)

# predicción
df_test_l3["l3"] = neigh.predict(X_pred_l3)

/tmp/ipykernel_84058/599255535.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test_l3["l3"] = neigh.predict(X_pred_l3)


In [19]:
df["l3"].loc[df["l3"].isnull()] = df_test_l3["l3"]

## Limpieza de otras columnas y lineas

In [20]:
df.drop(columns = ["ad_type","l4","l5", "l6", "created_on","start_date", "end_date","l1","l2","price_period","operation_type"], inplace = True )

In [21]:
df.dropna(subset=["price"], inplace = True)

df.dropna(subset=["currency"], inplace = True)

In [22]:
df = df.loc[df["currency"] == "USD"]

df.drop(columns = ["currency"], inplace = True )

/tmp/ipykernel_84058/3524703153.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns = ["currency"], inplace = True )


In [23]:
df["property_type_check"] = df.property_type.apply(lambda x: 1 if x in ("Departamento","PH") else 0)

df = df.loc[df["property_type_check"] == 1]

df.drop(columns = "property_type_check", inplace = True)

/tmp/ipykernel_84058/354315104.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["property_type_check"] = df.property_type.apply(lambda x: 1 if x in ("Departamento","PH") else 0)
/tmp/ipykernel_84058/354315104.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns = "property_type_check", inplace = True)


In [24]:
df.drop(df.loc[df["rooms"].isnull() & df["bedrooms"].isnull() & df["bathrooms"].isnull() & df["surface_total"].isnull() & df["surface_covered"].isnull()].index, inplace = True)

In [25]:
df_pred["rooms"].fillna(0, inplace = True)
df_pred["bedrooms"].fillna(0, inplace = True)
df_pred["bathrooms"].fillna(0, inplace = True)

In [26]:
df["rooms"].fillna(0, inplace = True)
df["bedrooms"].fillna(0, inplace = True)
df["bathrooms"].fillna(0, inplace = True)

In [27]:
df["surface_total"].loc[df["surface_total"].isnull() & df["surface_covered"] > 0] = df["surface_covered"].loc[df["surface_total"].isnull() & df["surface_covered"] > 0]

df["surface_total"].fillna(0, inplace = True)
df["surface_covered"].fillna(0, inplace = True)

/tmp/ipykernel_84058/3354645445.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["surface_total"].loc[df["surface_total"].isnull() & df["surface_covered"] > 0] = df["surface_covered"].loc[df["surface_total"].isnull() & df["surface_covered"] > 0]


In [28]:
df["bedrooms"].loc[df["bedrooms"] < 0 ] = abs(df["bedrooms"].loc[df["bedrooms"] < 0 ])

/tmp/ipykernel_84058/3611823614.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["bedrooms"].loc[df["bedrooms"] < 0 ] = abs(df["bedrooms"].loc[df["bedrooms"] < 0 ])


In [29]:
df["surface_total"].loc[df["surface_total"] < 0 ] = abs(df["surface_total"].loc[df["surface_total"] < 0 ])

/tmp/ipykernel_84058/3583135774.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["surface_total"].loc[df["surface_total"] < 0 ] = abs(df["surface_total"].loc[df["surface_total"] < 0 ])


In [30]:
df["surface_covered"].loc[df["surface_covered"] < 0 ] = abs(df["surface_covered"].loc[df["surface_covered"] < 0 ])

/tmp/ipykernel_84058/3604395483.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["surface_covered"].loc[df["surface_covered"] < 0 ] = abs(df["surface_covered"].loc[df["surface_covered"] < 0 ])


In [31]:
df.drop(columns = ["title"], inplace = True)

In [32]:
df.drop(columns = ["lat","lon"], inplace = True)

In [33]:
df.drop(df.loc[df["price"]> 6000000].index, inplace=True)

In [34]:
df.drop(df[df.price < 15000.0].index,inplace = True)

In [35]:
df.drop_duplicates(inplace = True)

## Modifico el df_pred

In [36]:
df_pred.drop(columns = ["ad_type","l4","l5", "l6", "created_on"], inplace = True )
df_pred.drop(columns = ["l1","l2"], inplace = True )
df_pred.drop(columns = ["currency"], inplace = True )
df_pred.drop(columns = ["price_period"], inplace = True)
df_pred.drop(columns = "operation_type", inplace = True)
df_pred.drop(columns = ["start_date","end_date","services"], inplace = True)
df_pred.drop(columns = ["title","details"], inplace = True)

In [37]:
df_pred.drop(columns = ["lat","lon"], inplace = True)

# Feature Engineering

Concateno los dataframes para que tengan las mismas columnas y las variables categoricas esten alineadas

In [38]:
df_total = pd.concat([df,df_pred], axis=0)

## Additionals

Genero dummies con los additionals y lo agrego en el df

In [39]:
df_pred.additionals.fillna("zxzyzjz", inplace = True)
df_pred.additionals_split = df_pred.additionals.apply(lambda x: x.split(","))

/tmp/ipykernel_84058/910923671.py:2: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df_pred.additionals_split = df_pred.additionals.apply(lambda x: x.split(","))


In [40]:
additionals_list = df_pred.additionals_split.explode().unique()
additionals_list = [x.strip() for x in additionals_list]
additionals_list = list(dict.fromkeys(additionals_list))

In [41]:
df_total.description.fillna("sin_descripcion", inplace = True)

In [42]:
for additional in additionals_list:
    df_total[additional] = df_total.description.apply(lambda x: 1 if additional.lower() in x.lower() else 0)

In [43]:
df_total.drop(columns = "zxzyzjz", inplace = True)
df_total.drop(columns = "additionals", inplace = True)
df_total.drop(columns = "description", inplace = True)

## logaritmicas

In [44]:
#df_total["lnrooms"] = df_total.rooms.apply(lambda x: x if x == 0 else np.log(x))

In [45]:
#df_total["lnbedrooms"] = df_total.bedrooms.apply(lambda x: x if x == 0 else np.log(x))

In [46]:
#df_total["lnsurface_covered"] = df_total.surface_covered.apply(lambda x: x if x == 0 else np.log(x))

In [47]:
#df_total["lnsurface_total"] = df_total.surface_total.apply(lambda x: x if x == 0 else np.log(x))

## Dummies

In [48]:
df_total = pd.get_dummies(df_total, columns = ["l3", "property_type"])

## Separo el dataframe nuevamente en df_pred y df

In [49]:
df = df_total[df_total.index.isin(df.index)]
df_pred = df_total[df_total.index.isin(df_pred.index)]

# Modelización

In [50]:
X = df[df.columns.drop('price')]
y = df['price']

## GBM

In [54]:
for n_estimators in [600, 700]:
    for max_depth in [5,7]:     

        gbm = sk.ensemble.GradientBoostingRegressor(n_estimators=n_estimators, max_depth=max_depth, random_state=42,learning_rate=0.08)

        scores = []
        kf = sk.model_selection.KFold(n_splits=5, shuffle=True, random_state=42)
        for train, test in kf.split(X):
            X_train, X_test, y_train, y_test = X.iloc[train], X.iloc[test], y[train], y[test]
        
            # Entrenamos el modelo
            gbm.fit(X_train, y_train)
        
            # Predecimos
            y_pred = gbm.predict(X_test)
        
            # Medimos la performance de la predicción
            score = sk.metrics.mean_squared_error(y_test, y_pred, squared=False)
            scores.append(score)
        print(f"n_{n_estimators} | max_depth:{max_depth} | mean: {np.mean(scores)} | sd: {np.std(scores)}")
        
    print("-----------------------------------------------------------------------------------")

n_600 | max_depth:5 | mean: 107782.14060053966 | sd: 3423.571159886552
n_600 | max_depth:7 | mean: 105277.80209686025 | sd: 3957.1942420357605
-----------------------------------------------------------------------------------
n_700 | max_depth:5 | mean: 107530.0206875155 | sd: 3530.653335203323
n_700 | max_depth:7 | mean: 105174.98604526161 | sd: 3837.959201673196
-----------------------------------------------------------------------------------


n_100 | max_depth:3 | mean: 123460.3142366433 | sd: 1727.6583321599176

n_100 | max_depth:5 | mean: 114657.89839219146 | sd: 2472.735674256593

n_300 | max_depth:3 | mean: 117398.20498211123 | sd: 2401.2053107628585

n_300 | max_depth:5 | mean: 109826.63624207438 | sd: 2983.8672972888517

n_500 | max_depth:3 | mean: 114691.00827011366 | sd: 2770.248958818647

n_500 | max_depth:5 | mean: 108405.9432217147 | sd: 3266.911700543427


## Predicción para Kaggle


In [55]:
# quedate con las mismas columnas que ar_properties
X_pred = df_pred[X.columns]

In [56]:
# los mejores hiperparámetros encontrados antes
n_estimators = 700
max_depth = 7

# Entrenamiento
gbm = sk.ensemble.GradientBoostingRegressor(n_estimators=n_estimators, max_depth=max_depth, random_state=42)

# Entrenamos el modelo
gbm.fit(X, y)

# predicción
df_pred["price"] = gbm.predict(X_pred)

/tmp/ipykernel_84058/2800229017.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pred["price"] = gbm.predict(X_pred)


In [57]:
# Grabo la solución
df_pred[["price"]].to_csv("../Soluciones/solucion.csv")